In [ ]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

from pyspark import SparkContext, SparkConf
import json
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "Lab3") 

spark = SparkSession.builder.config(conf=conf).getOrCreate()

import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, StringType
from pyspark.ml.feature import StringIndexer, VectorIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.ml.classification import LogisticRegression
import json
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField
from pyspark.ml.feature import CountVectorizer, StringIndexer, IndexToString
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.types import LongType


# Function to convert JSON array string to a list
def parse_json(array_str):
    json_obj = json.loads(array_str)
    for item in json_obj["visits"]:
        yield (item["url"], item["timestamp"])



In [ ]:
from pyspark import SparkContext, SparkConf
import json
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "Lab3") 
conf.set("spark.driver.memory", "16g") 
conf.set("spark.driver.memoryOverhead", "32g") 
conf.set("spark.executor.memory", "4g") 
conf.set("spark.executor.instances", "2") 

spark = SparkSession.builder.config(conf=conf).getOrCreate()




In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import MinMaxScaler
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, StringType
from pyspark.ml.feature import StringIndexer, VectorIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.ml.classification import LogisticRegression

items = spark.read.option("delimiter", "\t").csv("/labs/slaba03/laba03_items.csv", header = True)
test = spark.read.option("delimiter", ",").csv("/labs/slaba03/laba03_test.csv", header = True)
train = spark.read.option("delimiter", ",").csv("/labs/slaba03/laba03_train.csv", header = True)
views_programmes = spark.read.option("delimiter", ",").csv("/labs/slaba03/laba03_views_programmes.csv", header = True)

train = train.withColumn("item_id", F.col("item_id").cast(IntegerType()))
train = train.withColumn("user_id", F.col("user_id").cast(IntegerType()))
train = train.withColumn("purchase", F.col("purchase").cast(IntegerType()))

test = test.withColumn("item_id", F.col("item_id").cast(IntegerType()))
test = test.withColumn("user_id", F.col("user_id").cast(IntegerType()))
test = test.drop(F.col("purchase"))

als = ALS(maxIter=10, regParam=0.01, nonnegative = False, implicitPrefs=True, rank = 7,
          userCol="user_id", itemCol="item_id", ratingCol="purchase")

model = als.fit(train)
predictions = model.transform(test.fillna(0))


predictions = predictions.withColumnRenamed("prediction", "purchase")


predictions = predictions.toPandas()
predictions = predictions.sort_values(by=['user_id', 'item_id'])
print(predictions)
predictions.to_csv('lab03.csv')